In [ ]:
!pip install git+git://github.com/geopandas/geopandas.git &> /dev/null #the stuff after the & suppresses the code before it.

#need to clean up this dataset
import geopandas as gpd
geo = gpd.read_file('https://opendata.arcgis.com/datasets/ace8fa8bea514d07a3139e4657b3cd9c_0.geojson')

HTTPError: ignored

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter as amt

In [ ]:
geo = geo.drop(columns=['OBJECTID', 'GEO_ID', 'FIPS_CODE'])
colnames = ['county','state','white_only_total','wot_moe','black_only_total','bot_moe',
            'amer_indian_alaskanative_only_total','aiant_moe','asian_only_total','a_moe',
            'pacisland_only_total','pit_moe','other_only_total','other_moe','two_or_more_races_total',
            'tomr_moe','not_hislat_total','nothislat_moe','hislat_total','hislat_moe','geometry']
geo.columns = colnames

In [ ]:
geo.head(2)

,county,state,white_only_total,wot_moe,black_only_total,bot_moe,amer_indian_alaskanative_only_total,aiant_moe,asian_only_total,a_moe,pacisland_only_total,pit_moe,other_only_total,other_moe,two_or_more_races_total,tomr_moe,not_hislat_total,nothislat_moe,hislat_total,hislat_moe,geometry
0,Autauga County,Alabama,42437,337,10565,230,159,68,568,166,32,36,409,348,1030,250,53672,NaN,1528,NaN,"POLYGON ((-86.41312 32.70739, -86.41305 32.690..."
1,Baldwin County,Alabama,179526,747,19764,377,1522,337,1680,360,9,13,2034,783,3572,671,198754,NaN,9353,NaN,"POLYGON ((-87.76516 31.29735, -87.76361 31.297..."


In [ ]:
geo = geo.drop(columns = ['wot_moe','bot_moe','aiant_moe','a_moe','pit_moe','other_moe',
                    'tomr_moe','nothislat_moe','hislat_moe','geometry'])
geo.tail(2)

,county,state,white_only_total,black_only_total,amer_indian_alaskanative_only_total,asian_only_total,pacisland_only_total,other_only_total,two_or_more_races_total,not_hislat_total,hislat_total
3218,Kalawao County,Hawaii,16,2,2,21,19,0,15,73,2
3219,King County,Texas,226,0,0,0,0,2,0,190,38


This dataframe has a column dedicated to how many people in the population are not hispanic. Summing this with the amount of hispanics yields the total population of a county. This makes it easier to construct percentages if necessary.

In [ ]:
geo['total_pop'] = geo.not_hislat_total + geo.hislat_total

In [ ]:
geo['whiteonly_pct'] = geo.white_only_total/geo.total_pop
geo['blackonly_pct'] = geo.black_only_total/geo.total_pop
geo['indianonly_pct'] = geo.amer_indian_alaskanative_only_total/geo.total_pop
geo['asianonly_pct'] = geo.asian_only_total/geo.total_pop
geo['pacislandonly_pct'] = geo.pacisland_only_total/geo.total_pop
geo['otheronly_pct'] = geo.other_only_total/geo.total_pop
geo['twoplusonly_pct'] = geo.two_or_more_races_total/geo.total_pop
geo['hislatonly_pct'] = geo.hislat_total/geo.total_pop

In [ ]:
geo.isna().sum() #no missing values thank goodness.

county                                 0
state                                  0
white_only_total                       0
black_only_total                       0
amer_indian_alaskanative_only_total    0
asian_only_total                       0
pacisland_only_total                   0
other_only_total                       0
two_or_more_races_total                0
not_hislat_total                       0
hislat_total                           0
total_pop                              0
whiteonly_pct                          0
blackonly_pct                          0
indianonly_pct                         0
asianonly_pct                          0
pacislandonly_pct                      0
otheronly_pct                          0
twoplusonly_pct                        0
hislatonly_pct                         0
dtype: int64

In [ ]:
def countyStringClean(data):
  #this gets rid of unnecessary strings in the data set including " City" or " County"
  strings = data.county.tolist()
  badwords = [' City', ' County', ' Census Area', ' Municipio']

  correct = []
  idx = []

  for bad in badwords:
    for index, name in enumerate(strings):
      if bad in name:
        correct.append(name.replace(bad, ''))
        idx.append(index)

  for idx_idx, idx_val in enumerate(idx):
    strings[idx_val] = correct[idx_idx]
  
  data.county = strings
  return data

In [ ]:
klen = countyStringClean(geo)

In [ ]:
#remove 'states' not present in the orig df
klen = klen[klen["state"].str.contains("Puerto Rico") == False]
klen = klen[klen["state"].str.contains("District of Columbia") == False]

In [ ]:
ogdf = pd.read_csv('/content/olsdf_wdum.csv')
ogdf.head(2)

,pos,county,state,pc_income,medhouse_inc,medfam_inc,pop,households,exp_rate,NE_dum,MW_dum,W_dum,S_dum
0,0,Arlington,Virginia,62018.0,103208.0,139244.0,214861.0,94454.0,0.067843,0,0,0,1
1,1,Marin,California,56791.0,90839.0,117357.0,254643.0,102912.0,0.087517,0,0,1,0


In [ ]:
klen.head(2)

,county,state,white_only_total,black_only_total,amer_indian_alaskanative_only_total,asian_only_total,pacisland_only_total,other_only_total,two_or_more_races_total,not_hislat_total,hislat_total,total_pop,whiteonly_pct,blackonly_pct,indianonly_pct,asianonly_pct,pacislandonly_pct,otheronly_pct,twoplusonly_pct,hislatonly_pct
0,Autauga,Alabama,42437,10565,159,568,32,409,1030,53672,1528,55200,0.768786,0.191395,0.002880,0.010290,0.000580,0.007409,0.018659,0.027681
1,Baldwin,Alabama,179526,19764,1522,1680,9,2034,3572,198754,9353,208107,0.862662,0.094970,0.007314,0.008073,0.000043,0.009774,0.017164,0.044943


In [ ]:
print('the shape of the original dataframe is,',ogdf.shape,'while the shape of the dataframe i want to append is,',klen.shape)

the shape of the original dataframe is, (3042, 13) while the shape of the dataframe i want to append is, (3141, 20)


In [ ]:
klen.index = range(0,len(klen.index))

In [ ]:
mergeddf = ogdf.merge(klen, on = ['state','county'], how = 'left')

In [ ]:
mergeddf[mergeddf['whiteonly_pct'].isna()]

,pos,county,state,pc_income,medhouse_inc,medfam_inc,pop,households,exp_rate,NE_dum,MW_dum,W_dum,S_dum,white_only_total,black_only_total,amer_indian_alaskanative_only_total,asian_only_total,pacisland_only_total,other_only_total,two_or_more_races_total,not_hislat_total,hislat_total,total_pop,whiteonly_pct,blackonly_pct,indianonly_pct,asianonly_pct,pacislandonly_pct,otheronly_pct,twoplusonly_pct,hislatonly_pct
43,43,James City,Virginia,39133.0,76960.0,90406.0,68171.0,26883.0,0.035732,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,86,Anchorage,Alaska,36214.0,77454.0,90466.0,295237.0,105208.0,0.069475,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,284,St. Tammany,Louisiana,30363.0,60799.0,74834.0,236832.0,87518.0,0.069357,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,331,Cameron,Louisiana,29559.0,64574.0,71310.0,6789.0,2529.0,0.079478,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,375,Lafayette,Louisiana,28901.0,51462.0,67061.0,224719.0,86504.0,0.060512,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855,2855,Winn,Louisiana,16789.0,34322.0,45200.0,15129.0,5402.0,0.074959,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2865,2865,Morehouse,Louisiana,16683.0,28585.0,39570.0,27603.0,10482.0,0.064334,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2889,2889,Concordia,Louisiana,16431.0,29022.0,38125.0,20652.0,7642.0,0.040650,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,3011,Madison,Louisiana,13585.0,25498.0,33063.0,12049.0,4068.0,0.104230,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mergeddf = mergeddf.dropna()

In [ ]:
mergeddf.shape

(2978, 31)

In [ ]:
mergeddf.to_csv(r'/content/covid_exp.csv', index = False)

In [ ]:
df = pd.read_csv('/content/covid_exp.csv')

In [ ]:
df[df['state'] == 'California'].shape

(58, 31)

stuff after this section adds the population density and amount of people per household. THANKS DR. SHILLING

In [ ]:
geo1 = gpd.read_file('https://opendata.arcgis.com/datasets/21843f238cbb46b08615fc53e19e0daf_1.geojson')
geo1.head(3)

,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area,geometry
0,1,00161526,01001,1.539602e+09,2.570696e+07,Autauga County,Alabama,2.59,0.05,2.59,0.07,2.60,0.16,55200,NaN,35.853419,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,2.066037,0.150256,"MULTIPOLYGON (((-86.41312 32.70739, -86.41305 ..."
1,2,00161527,01003,4.117547e+09,1.133056e+09,Baldwin County,Alabama,2.61,0.04,2.66,0.06,2.48,0.11,208107,NaN,50.541504,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,4.483746,0.409904,"MULTIPOLYGON (((-87.76516 31.29735, -87.76361 ..."
2,3,00161528,01005,2.292145e+09,5.053870e+07,Barbour County,Alabama,2.49,0.07,2.44,0.11,2.58,0.15,25782,NaN,11.247981,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,2.695262,0.223270,"MULTIPOLYGON (((-85.05603 32.06306, -85.05593 ..."


In [ ]:
geo1 = geo1.drop(columns = ['FID', 'COUNTYNS', 'GEOID', 'ALAND', 'AWATER'])
geo1.head(3)

,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area,geometry
0,Autauga County,Alabama,2.59,0.05,2.59,0.07,2.60,0.16,55200,NaN,35.853419,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,2.066037,0.150256,"MULTIPOLYGON (((-86.41312 32.70739, -86.41305 ..."
1,Baldwin County,Alabama,2.61,0.04,2.66,0.06,2.48,0.11,208107,NaN,50.541504,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,4.483746,0.409904,"MULTIPOLYGON (((-87.76516 31.29735, -87.76361 ..."
2,Barbour County,Alabama,2.49,0.07,2.44,0.11,2.58,0.15,25782,NaN,11.247981,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,2.695262,0.223270,"MULTIPOLYGON (((-85.05603 32.06306, -85.05593 ..."


In [ ]:
colnames = ['county','state','avg_house','avg_hosue_moe','avg_oou','avg_oou_moe',
            'avg_house_rent', 'ahr_moe', 'total_pop', 'tot_pop_moe', 'pop_density', 'user', 'date',
            'edit_user', 'edit_date', 'shape_area', 'shape_len', 'pop_dens_moe', 'poop']
geo1.columns = colnames

In [ ]:
geo1.head(2)

,county,state,avg_house,avg_hosue_moe,avg_oou,avg_oou_moe,avg_house_rent,ahr_moe,total_pop,tot_pop_moe,pop_density,user,date,edit_user,edit_date,shape_area,shape_len,pop_dens_moe,poop
0,Autauga County,Alabama,2.59,0.05,2.59,0.07,2.60,0.16,55200,NaN,35.853419,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,2.066037,0.150256,"MULTIPOLYGON (((-86.41312 32.70739, -86.41305 ..."
1,Baldwin County,Alabama,2.61,0.04,2.66,0.06,2.48,0.11,208107,NaN,50.541504,esri_demographics,2020-04-01T20:53:36,esri_demographics,2020-04-01T20:53:36,0.0,4.483746,0.409904,"MULTIPOLYGON (((-87.76516 31.29735, -87.76361 ..."


In [ ]:
newdf = geo1[['county','state','avg_house','pop_density']]
newdf.head(2)

,county,state,avg_house,pop_density
0,Autauga County,Alabama,2.59,35.853419
1,Baldwin County,Alabama,2.61,50.541504


In [ ]:
klendf = countyStringClean(newdf)
klendf.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,county,state,avg_house,pop_density
0,Autauga,Alabama,2.59,35.853419
1,Baldwin,Alabama,2.61,50.541504


In [ ]:
klendf.shape

(3220, 4)

In [ ]:
df = pd.read_csv('/content/covid_exp.csv')
df.head(2)

,pos,county,state,pc_income,medhouse_inc,medfam_inc,pop,households,exp_rate,NE_dum,MW_dum,W_dum,S_dum,white_only_total,black_only_total,amer_indian_alaskanative_only_total,asian_only_total,pacisland_only_total,other_only_total,two_or_more_races_total,not_hislat_total,hislat_total,total_pop,whiteonly_pct,blackonly_pct,indianonly_pct,asianonly_pct,pacislandonly_pct,otheronly_pct,twoplusonly_pct,hislatonly_pct
0,0,Arlington,Virginia,62018.0,103208.0,139244.0,214861.0,94454.0,0.067843,0,0,0,1,165609.0,20777.0,870.0,23992.0,260.0,11833.0,8462.0,195623.0,36180.0,231803.0,0.714439,0.089632,0.003753,0.103502,0.001122,0.051048,0.036505,0.156081
1,1,Marin,California,56791.0,90839.0,117357.0,254643.0,102912.0,0.087517,0,0,1,0,203342.0,5709.0,737.0,15000.0,584.0,22262.0,12661.0,218960.0,41335.0,260295.0,0.781198,0.021933,0.002831,0.057627,0.002244,0.085526,0.048641,0.158801


In [ ]:
df_merge = df.merge(klendf, on = ['state','county'], how = 'left')
df_merge.head(5)

,pos,county,state,pc_income,medhouse_inc,medfam_inc,pop,households,exp_rate,NE_dum,MW_dum,W_dum,S_dum,white_only_total,black_only_total,amer_indian_alaskanative_only_total,asian_only_total,pacisland_only_total,other_only_total,two_or_more_races_total,not_hislat_total,hislat_total,total_pop,whiteonly_pct,blackonly_pct,indianonly_pct,asianonly_pct,pacislandonly_pct,otheronly_pct,twoplusonly_pct,hislatonly_pct,avg_house,pop_density
0,0,Arlington,Virginia,62018.0,103208.0,139244.0,214861.0,94454.0,0.067843,0,0,0,1,165609.0,20777.0,870.0,23992.0,260.0,11833.0,8462.0,195623.0,36180.0,231803.0,0.714439,0.089632,0.003753,0.103502,0.001122,0.051048,0.036505,0.156081,2.18,3442.635855
1,1,Marin,California,56791.0,90839.0,117357.0,254643.0,102912.0,0.087517,0,0,1,0,203342.0,5709.0,737.0,15000.0,584.0,22262.0,12661.0,218960.0,41335.0,260295.0,0.781198,0.021933,0.002831,0.057627,0.002244,0.085526,0.048641,0.158801,2.40,193.100506
2,2,Pitkin,Colorado,51814.0,72745.0,93981.0,17173.0,7507.0,0.022957,0,0,1,0,16316.0,297.0,22.0,270.0,0.0,749.0,255.0,16132.0,1777.0,17909.0,0.911050,0.016584,0.001228,0.015076,0.000000,0.041823,0.014239,0.099224,2.53,7.123410
3,3,Los Alamos,New Mexico,51044.0,106686.0,124979.0,17979.0,7590.0,0.032179,0,0,1,0,15686.0,36.0,201.0,996.0,0.0,567.0,870.0,15120.0,3236.0,18356.0,0.854543,0.001961,0.010950,0.054260,0.000000,0.030889,0.047396,0.176291,2.41,64.901861
4,4,Fairfax,Virginia,50532.0,110292.0,128596.0,1101071.0,389908.0,0.100812,0,0,0,1,708761.0,111026.0,2507.0,217066.0,838.0,50267.0,53064.0,957978.0,185551.0,1143529.0,0.619802,0.097091,0.002192,0.189821,0.000733,0.043958,0.046404,0.162262,2.88,1129.205805


In [ ]:
df_merge.to_csv(r'/content/final_covid_pls.csv', index = False)